# 2025B · Unificación de Observaciones y Métricas

Une **2025B completo** con:
- **Métricas** (`cls_A`, `cls_B`, `diferencia`, `Observaciones_HorasClase`)
- **Actividades** (dos columnas largas + `Observaciones_Actividades1:1`)

Prioridad de emparejamiento: **Identificación → Profesor → Profesor.1**. Exporta un Excel filtrable con encabezado congelado y *wrap text* en las observaciones.


## 1) Configuración

In [1]:

import pandas as pd
import re, unicodedata, os

excel_b_full   = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Unificacion\data\2025B.xlsx"
excel_metricas = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Unificacion\data\Reporte_B_menor_A.xlsx"
excel_activ    = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Unificacion\data\Reporte_actividades_3_4.xlsx"
out_dir        = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Unificacion\outputs_notebook"
out_xlsx       = "2025B_Observaciones.xlsx"


## 2) Parámetros y columnas objetivo

In [2]:

ID_H      = "Identificación"
PROF_H    = "Profesor"   # existen 'profesor' y 'profesor.1'
CLS_A_H   = "cls_A"
CLS_B_H   = "cls_B"
DIF_H     = "diferencia"

OBS_MET_H = "Observaciones_HorasClase"        # desde métricas
OBS_ACT_H = "Observaciones_Actividades1:1"    # desde actividades

TAREA_1_H = "Preparación y actualización de clases, seminarios, talleres, entre otros"
TAREA_2_H = "Preparación, elaboración, aplicación y calificación de exámenes, trabajos y prácticas; consultas académicas"

BASE_B_ORDER = [
    "row_index","periodo academico","departamento","tipo cargo","dedicacion",
    "profesor","identificacion","profesor.1","tipo","cargo","dedicacion.1",
    "fecha inicio","fecha fin","estado","horas exigibles","cls-dictadas"
]


## 3) Utilidades

In [3]:

def _norm_text(s: str) -> str:
    """acentos fuera, minúsculas, recortar"""
    return "".join(ch for ch in unicodedata.normalize("NFKD", str(s)) if not unicodedata.combining(ch)).lower().strip()

def norm_ident(v: str) -> str:
    s = _norm_text(v).replace(" ", "").replace("\u00A0", "")
    digits = re.sub(r"\D", "", s)
    return digits if len(digits) >= 8 else re.sub(r"[^0-9a-z]", "", s)

def norm_prof(s: str) -> str:
    s = _norm_text(s)
    return re.sub(r"\s+", " ", s)

def read_master_all_sheets(path: str) -> pd.DataFrame:
    """Lee todas las hojas de un Excel como dtype=object; normaliza encabezados;
    añade '_sheet' y 'row_index'."""
    xls = pd.ExcelFile(path)  # requiere openpyxl
    frames = []
    for sh in xls.sheet_names:
        df = pd.read_excel(path, sheet_name=sh, dtype="object")
        df.columns = [_norm_text(c) for c in df.columns]
        df["_sheet"] = sh
        frames.append(df)
    out = pd.concat(frames, ignore_index=True)
    out.insert(0, "row_index", out.index.astype(int))
    return out

def resolve_all(df: pd.DataFrame, human: str):
    tgt = _norm_text(human)
    return [c for c in df.columns if tgt in c]

def resolve_best(df: pd.DataFrame, human: str) -> str:
    """Match exacto (normalizado); si no, substring."""
    tgt = _norm_text(human)
    exact = [c for c in df.columns if c == tgt]
    if exact: return exact[0]
    subs = [c for c in df.columns if tgt in c]
    if subs: return subs[0]
    raise ValueError(f"Columna '{human}' no encontrada (tgt='{tgt}')")

def map_from_df(df: pd.DataFrame, key_col: str, value_col: str):
    if not key_col or not value_col or key_col not in df or value_col not in df:
        return {}
    s = df[[key_col, value_col]].dropna(subset=[key_col])
    return dict(zip(s[key_col], s[value_col]))

def triple_map(series_ident, series_prof1, series_prof2, d_ident, d_prof):
    """Prioridad: ident → prof1 → prof2."""
    n = len(series_ident)
    s_id = series_ident.map(d_ident) if d_ident else pd.Series([pd.NA]*n)
    s_p1 = series_prof1.map(d_prof)  if d_prof  else pd.Series([pd.NA]*n)
    s_p2 = series_prof2.map(d_prof)  if d_prof  else pd.Series([pd.NA]*n)
    out = s_id.where(s_id.notna(), s_p1)
    out = out.where(out.notna(), s_p2)
    return out

def pick_cols_in_order(df: pd.DataFrame, wanted):
    """Devuelve, en orden, las columnas de 'wanted' que existan en df (exacto o substring)."""
    cols = []
    for w in wanted:
        tgt = _norm_text(w)
        if tgt in df.columns:
            cols.append(tgt); continue
        found = next((c for c in df.columns if tgt in c), None)
        if found: cols.append(found)
    return cols


## 4) Cargar 2025B y preparar claves

In [ ]:

B = read_master_all_sheets(excel_b_full)

# columnas clave en B
id_b = resolve_best(B, ID_H)
prof_cols_b = resolve_all(B, PROF_H)  
prof_1 = prof_cols_b[0] if len(prof_cols_b) >= 1 else None
prof_2 = prof_cols_b[1] if len(prof_cols_b) >= 2 else None

# claves normalizadas
B["key_ident"] = B[id_b].map(norm_ident)
B["key_prof1"] = B[prof_1].map(norm_prof) if prof_1 else pd.NA
B["key_prof2"] = B[prof_2].map(norm_prof) if prof_2 else pd.NA

B.head(3)


c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,row_index,periodo academico,departamento,tipo cargo,dedicacion,profesor,identificacion,profesor.1,tipo,cargo,...,total horas actividades de gestion,vacaciones,permisos,total horas vacaciones y permisos,total horas d + i + g + p v,total horas/exigibles,_sheet,key_ident,key_prof1,key_prof2
0,0,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0401337951/ NOMBRE: FLORES BEN...,0401337951,FLORES BENALCAZAR CARMEN JOHANNA,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,NaN,128,NaN,128,976,0,Export,0401337951,identificacion: 0401337951/ nombre: flores ben...,flores benalcazar carmen johanna
1,1,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0401545843/ NOMBRE: MONTENEGRO...,0401545843,MONTENEGRO SALAZAR BYRON ESTEBAN,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,NaN,168,NaN,168,976,0,Export,0401545843,identificacion: 0401545843/ nombre: montenegro...,montenegro salazar byron esteban
2,2,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0503489809/ NOMBRE: CHILLA PRI...,0503489809,CHILLA PRIETO MARIO FERNANDO,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,NaN,104,NaN,104,976,0,Export,0503489809,identificacion: 0503489809/ nombre: chilla pri...,chilla prieto mario fernando


## 5) Cargar métricas y construir mapeos

In [5]:

M = pd.read_excel(excel_metricas, dtype="object")
M.columns = [_norm_text(c) for c in M.columns]

mid   = resolve_best(M, ID_H)               if resolve_all(M, ID_H)   else None
mpro  = resolve_best(M, PROF_H)             if resolve_all(M, PROF_H) else None
ma    = resolve_best(M, CLS_A_H)            if resolve_all(M, CLS_A_H) else None
mb    = resolve_best(M, CLS_B_H)            if resolve_all(M, CLS_B_H) else None
md    = resolve_best(M, DIF_H)              if resolve_all(M, DIF_H)   else None
mobs  = resolve_best(M, "Observaciones")    if resolve_all(M, "Observaciones") else None

if mid:  M["key_ident"] = M[mid].map(norm_ident)
if mpro: M["key_prof"]  = M[mpro].map(norm_prof)

dict_clsA_ident = map_from_df(M, "key_ident", ma)
dict_clsB_ident = map_from_df(M, "key_ident", mb)
dict_dif_ident  = map_from_df(M, "key_ident", md)
dict_obs_ident  = map_from_df(M, "key_ident", mobs)

dict_clsA_prof = map_from_df(M, "key_prof", ma)
dict_clsB_prof = map_from_df(M, "key_prof", mb)
dict_dif_prof  = map_from_df(M, "key_prof", md)
dict_obs_prof  = map_from_df(M, "key_prof", mobs)


## 6) Cargar actividades y construir mapeos

In [6]:

A2 = pd.read_excel(excel_activ, dtype="object")
A2.columns = [_norm_text(c) for c in A2.columns]

aid   = resolve_best(A2, ID_H)                  if resolve_all(A2, ID_H)   else None
apro  = resolve_best(A2, PROF_H)                if resolve_all(A2, PROF_H) else None
ac1   = resolve_best(A2, TAREA_1_H)             if resolve_all(A2, TAREA_1_H) else None
ac2   = resolve_best(A2, TAREA_2_H)             if resolve_all(A2, TAREA_2_H) else None
aobs  = resolve_best(A2, "Observaciones")       if resolve_all(A2, "Observaciones") else None

if aid:  A2["key_ident"] = A2[aid].map(norm_ident)
if apro: A2["key_prof"]  = A2[apro].map(norm_prof)

dict_c1_ident = map_from_df(A2, "key_ident", ac1)
dict_c2_ident = map_from_df(A2, "key_ident", ac2)
dict_oa_ident = map_from_df(A2, "key_ident", aobs)

dict_c1_prof = map_from_df(A2, "key_prof", ac1)
dict_c2_prof = map_from_df(A2, "key_prof", ac2)
dict_oa_prof = map_from_df(A2, "key_prof", aobs)


## 7) Aplicar a 2025B ( Ident → Profesor → Profesor.1)

In [2]:

# Métricas
B[CLS_A_H] = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_clsA_ident, dict_clsA_prof)
B[CLS_B_H] = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_clsB_ident, dict_clsB_prof)
B[DIF_H]   = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_dif_ident,  dict_dif_prof)

# Actividades
B[TAREA_1_H] = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_c1_ident, dict_c1_prof)
B[TAREA_2_H] = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_c2_ident, dict_c2_prof)

# Observaciones separadas
obs_m = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_obs_ident, dict_obs_prof)   # horas clase
obs_a = triple_map(B["key_ident"], B["key_prof1"], B["key_prof2"], dict_oa_ident,  dict_oa_prof)    # actividades

B[OBS_MET_H] = obs_m
B[OBS_ACT_H] = obs_a

B[[CLS_A_H, CLS_B_H, DIF_H, TAREA_1_H, TAREA_2_H, OBS_ACT_H, OBS_MET_H]].head(3)


NameError: name 'triple_map' is not defined

## 8) Selección final y exportación

In [8]:

# Columnas base reales de B
base_cols = pick_cols_in_order(B, BASE_B_ORDER)

# Eliminar ayudantes temporales
drop_helpers = [c for c in ["_sheet","key_ident","key_prof1","key_prof2"] if c in B.columns]
B_final = B.drop(columns=drop_helpers, errors="ignore")

# Orden final
final_cols = base_cols + [
    CLS_A_H, CLS_B_H, DIF_H,
    TAREA_1_H, TAREA_2_H,
    OBS_ACT_H, OBS_MET_H
]
final_existing = [c for c in final_cols if c in B_final.columns]
B_final = B_final[final_existing].copy()

# Exportar
os.makedirs(out_dir, exist_ok=True)
xlsx_path = os.path.join(out_dir, out_xlsx)

with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xl:
    B_final.to_excel(xl, sheet_name="2025B_Observaciones", index=False)

    from openpyxl.styles import Font, Alignment
    wb = xl.book; ws = wb["2025B_Observaciones"]

    # encabezado + filtros + freeze
    for cell in next(ws.iter_rows(min_row=1, max_row=1)):
        cell.font = Font(bold=True)
        cell.alignment = Alignment(horizontal="center")
    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = "A2"

    # anchos
    for col in ws.columns:
        maxlen = max((len(str(c.value)) if c.value is not None else 0) for c in col)
        ws.column_dimensions[col[0].column_letter].width = min(max(12, maxlen + 2), 60)

    # wrap text en observaciones
    headers = {c.value: i+1 for i, c in enumerate(ws[1])}
    for col_name in [OBS_ACT_H, OBS_MET_H]:
        if col_name in headers:
            col_letter = ws.cell(row=1, column=headers[col_name]).column_letter
            for cell in ws[col_letter][1:]:
                cell.alignment = Alignment(wrap_text=True, vertical="top")

print(f"OK. Excel de salida: {xlsx_path}")
B_final.head(10)


OK. Excel de salida: C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Unificacion\outputs_notebook\2025B_Observaciones.xlsx


,row_index,periodo academico,departamento,tipo cargo,dedicacion,profesor,identificacion,profesor.1,tipo,cargo,...,estado,horas exigibles,cls-dictadas,cls_A,cls_B,diferencia,"Preparación y actualización de clases, seminarios, talleres, entre otros","Preparación, elaboración, aplicación y calificación de exámenes, trabajos y prácticas; consultas académicas",Observaciones_Actividades1:1,Observaciones_HorasClase
0,0,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0401337951/ NOMBRE: FLORES BEN...,0401337951,FLORES BENALCAZAR CARMEN JOHANNA,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,342,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0401545843/ NOMBRE: MONTENEGRO...,0401545843,MONTENEGRO SALAZAR BYRON ESTEBAN,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0503489809/ NOMBRE: CHILLA PRI...,0503489809,CHILLA PRIETO MARIO FERNANDO,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0604078196/ NOMBRE: SANCHEZ MI...,0604078196,SANCHEZ MIRANDA XIMENA JHOANA,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,No registra horas de Preparación y actualizaci...,NaN
4,4,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0705460624/ NOMBRE: TORRES TOR...,0705460624,TORRES TORRES MARIBEL CAROLINA,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,No registra horas de Preparación y actualizaci...,NaN
5,5,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0803014844/ NOMBRE: CAICEDO OR...,0803014844,CAICEDO ORTIZ GANDHY,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 1004037816/ NOMBRE: PERUGACHI ...,1004037816,PERUGACHI BETANCOURT TELMO ALFREDO,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,378,NaN,NaN,NaN,NaN,NaN,No registra horas de Preparación y actualizaci...,NaN
7,7,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 1311967101/ NOMBRE: LOPEZ ARIA...,1311967101,LOPEZ ARIAS ALISSON LORENA,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 1709767667/ NOMBRE: BASANTES M...,1709767667,BASANTES MORENO FABIAN WLADIMIRO,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2025-B,DEPARTAMENTO DE FORMACION BASICA,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 1710962026/ NOMBRE: PACHACAMA ...,1710962026,PACHACAMA MOROCHO RAMON EDISON,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,...,REVISION INICIAL,976,360,414.0,360.0,54.0,NaN,NaN,No registra horas de Preparación y actualizaci...,Se observa una disminución en la carga\nhorari...
